# Deployment -- On Azure

In this notebook, we continue to deploy our AI model as a Webservice.  
As our AI model has been trained and registered in the previous notebook, we can easily load in this information in here!

**SELECT THE RIGHT KERNELS**

In [ ]:
model_name = 'animal-cnn'

In [ ]:
import os
import cv2

In [ ]:
## Import AzureML packages
from azureml.core import Workspace
from azureml.core import Dataset
from azureml.core import Model
from azureml.data.datapath import DataPath
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

In [ ]:
import json
import numpy as np
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

## Step 1: Connect Workspace

In [ ]:
## Either get environment variables, or a fallback name, which is the second parameter.
## Currently, fill in the fallback values. Later on, we will make sure to work with Environment values. So we're already preparing for it in here!
workspace_name = os.environ.get('WORKSPACE', 'MLOps-Workshop')
subscription_id = os.environ.get('SUBSCRIPTION_ID', '763622cd-d9e1-46f1-84c7-635df9708641')
resource_group = os.environ.get('RESOURCE_GROUP', 'TETRA-Workshop-2406')

In [ ]:
ws = Workspace.get(name=workspace_name,
               subscription_id=subscription_id,
               resource_group=resource_group)

## Step 2: Create a deployment script and environment

In [ ]:
%%writefile scripts/score.py
import os
import numpy as np
import json
from tensorflow import keras
from tensorflow.keras.models import load_model
from PIL import Image

ANIMALS = ['Cat', 'Dog', 'Panda']

def init():
    global model

    # The AZUREML_MODEL_DIR environment variable indicates
    # a directory containing the model file you registered.
    model_path = os.path.join(os.environ.get('AZUREML_MODEL_DIR'), 'animal-cnn-test')

    model = load_model(model_path)

def run(image):
    data = json.loads(image)
    img = np.asarray(data['data'])
    print(img.shape)
    images_to_predict = np.expand_dims(img, axis=0)
    predictions = model.predict(images_to_predict)
    classifications = predictions.argmax(axis=1)

    return ANIMALS[classifications.tolist()[0]]

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

environment_name = os.environ.get('TRAINING_ENV_NAME', 'animals-classification-env-deployment')
environment = Environment(environment_name)
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'tensorflow',
    'numpy',
    'Pillow'
])

In [ ]:
print()

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = os.environ.get('SCRIPT_SERVICE_NAME', 'animals-classification-svc-3')

inference_config = InferenceConfig(entry_script='scripts/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Get our model based on the name we registered in the previous notebook
model = Model(ws, model_name)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

## Step 3 -- Test the service

Now that we have an endpoint, we can try to upload an image and get a result.  
We will just get a value of 0 - 2, where 0 == 'Cats', 1 == 'Dogs' and 2 == 'Pandas'.  

I found these values in the Logs of our AI model training, but it's also the order we have always used.  
```text
# Logging information
...
['cats' 'dogs' 'panda'] -- [0 1 2]
...
```

In [ ]:
# Read in a test image
test_image = cv2.imread('data/animals/cats/cats_00001.jpg')
test_image = cv2.resize(test_image, (64, 64))

In [ ]:
service.run(json.dumps({'data': test_image}, cls=NumpyEncoder))

## Step 3b -- Test the service with default Python requests 

In [ ]:
# URL for the web service
scoring_uri = service.scoring_uri
print(scoring_uri)

In [ ]:
import requests
import json

# Two sets of data to score, so we get two results back
data = {"data": test_image}
# Convert to JSON string
input_data = json.dumps(data, cls=NumpyEncoder)

# Set the content type
headers = {'Content-Type': 'application/json'}

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.text)

## Step 4 -- Clear the service

Execute this cell to remove your service after it has been tested.

In [ ]:
service.delete()